Before I start, I want to acknowledge that due to my lack of prerequisite knowledge in neural networks and LLMs, I have heavily relied on external resources to complete this task, mainly from [Qiskit Ecosystem](https://qiskit-community.github.io/qiskit-machine-learning/) which has multiple [contributors](https://github.com/qiskit-community/qiskit-machine-learning/graphs/contributors) working on quantum machine learning.

I recommend checking out their work for more in-depth understanding and additional resources on the topic, as well as a soft introduction to [PyTorch](../PyTorch) in my Github. Don't worry if it seems overwhelming at first; with time and practice, you'll get the hang of it! I'll introduce the basic concepts and provide code examples to help you get started with quantum machine learning using Qiskit. I'll also provid a primer on neural networks and large language models (LLMs) to give you a foundational understanding before diving into the quantum aspects.

# To Kickstart; Machine Learning

What is Machine Learning? Why does everyone and their grandma seem to be talking about it? In simple terms, it's like teaching a computer to recognize patterns and make informed choices based on those patterns. The simplest example of this in practice is spam email detection in mail services like Gmail, and generative AI tools like ChatGPT. It does this by implementing, surely enough, algorithms to data sets.

Moving up a level of difficulty, I can present the formal definition; Machine Learning is a subset of AI that deals with the development of statistical algorithms and models that can learn from 'training data' to make predictions on 'unseen data', thus perfoming tasks without being explicitly programmed for each specific task. As you can guess, a strong background and understanding of statistics and mathematical optimization is required to fully grasp the concepts of machine learning.

Now, this isn't a machine learning course, so I won't bore you with all the nitty-gritty details. Instead, I'll define exactly what Quantum Machine Learning (QML) is, and how it differs from classical machine learning.

It's understandable that one might think that QML describes quantum computers engaged in machine learning tasks, but it's more nuanced than that. This definition is quite narrow and limiting, as we as physics enthusiasts are naturally more concerned with employing machine learning techniques to **solve quantum problems** and **enhance quantum computing capabilities**.

Please keep this distinction in mind as we proceed. Quantum Machine Learning is a multidisciplinary field that combines principles from quantum computing and machine learning to develop algorithms that can leverage the unique properties of quantum mechanics to work towards quantum supremacy in machine learning tasks.

# Add the 'Quantum' in there:

A generic problem can be solved for the first time on a quantum computer, and then subsequently people find a way to do it on a classical supercomputer. This cycle of classical and quantum computing pushing each other to their limits will likely go on and on. There are specific problems where quantum supremacy is achievable, given progress in areas such as reduction of errors and the number of available physical qubits.

QML might plausibly include quantum mechanics in either the data or algorithmic sides, or both. We'll mostly restrict ourselves to discussions of **quantum algorithms** applied to **classical data.** One reason for this is that ML problems with classical data are already so well studied and widely available, and another reason is the lack of QRAM. Without the ability to store large amounts of quantum data on a relatively long timescale, methods that begin with quantum data are still fairly far from applicability to industry. It is also unclear how to "quantumly-access" classical data in an efficient manner.

Quantum algorithms can offer an exponential speed-up over classical algorithms for **very specific problems** that are classically hard, and well-suited to quantum algorithms. They aren't a magic solution for all machine learning problems. Higher dimensionality of quantum states can potentially allow for more complex data representations, which might help in capturing intricate patterns in data, according to Cover's Theorem:

> "A complex pattern-classification problem, cast in a high-dimensional space nonlinearly, is more likely to be linearly separable than in a low-dimensional space."

A common goal in QML is to find a mapping from the lower-dimensional set of features into a higher-dimensional space, that effectively separates datapoints so we can use the mapping to classify new datapoints. However, increased dimensions in quantum computing cannot be a sufficient condition for computational power over classical alternatives, and there are nuances and caveats to consider.

You might recall that I mentioned the Hilbert Space not too long ago in my introductory notebook. This space, being exponentially larger for quantum systems compared to classical ones, indeed provides a fertile ground for representing complex data structures. Let's try to mathematically explore the implications. 

Cover's theorem in principle, suggests that for a set of real datapoints $x_i\in\mathbb R^d$, where $d$ is a given dimension, there exists a non-linear mapping $$\zeta:\mathbb R^d\to\mathbb R^D$$ into a higher-dimensional space such that the transformed points $\zeta(x_i)$ are linearly separable with high probability, provided that $D>>d$. Also, the probability that some binary labels $y_i\in\{-1,1\}$ associated with the points $x_i$ can be separated by a hyperplane in the 
$\zeta$-space grows rapidly with $D$.

Since quantum computers can represent data in a Hilbert space of dimension $2^n$ for $n$ qubits, mapping a quantum state $\ket{\zeta}$ to a higher-dimensional space is inherently feasible, through what's called a *Quantum Feature Map.* This map encodes classical data into quantum states, and establishes mathematical continuity with Cover's theorem and ML:

$$\begin{align*}\ket{\zeta(x)} = \hat U_\zeta(x)\ket{0}^{\otimes n}&&x\to\ket{\zeta(x)}\in\mathbb C^{2^n}\end{align*}$$

# So how does Qiskit help?

What 'map' I described earlier is implementd in QML through a *Quantum Kernel Method.* For each classical datapoint $x_i$ you choose a parameterized unitary $\hat U$ embedding:

$$\ket{\zeta(x_i)} = \hat U(x_i)\ket{0}^{\otimes n}$$

A *Kernel Function* $K(x_i,x_j)$ computes the inner product between two quantum states. Why? It tells you how similar every pair of data points is, according to some meaningful similarity measure, similiar to classical ML. Think of it like a separator that helps you distinguish between different classes of data points. These functions make up the *Kernel Matrix,* whose elements are

$$K_{ij}\equiv K(x_i,x_j) = |\langle \zeta(x_i)|\zeta(x_j)\rangle|^2$$

To recap terminology, $\zeta$ is the quantum feature map that encodes classical data into quantum states, and $K$ is the kernel function that computes the similarity between pairs of these quantum states. The kernel matrix $K_{ij}$ contains all pairwise similarities, and is an inner product matrix in the quantum feature space of $\dim(\mathcal H)=2^n$.

If you're interested in diving deeper into how inner products are computed on quantum computers, I recommend checking out the [Qiskit Machine Learning documentation](https://qiskit-community.github.io/qiskit-machine-learning/) for more detailed explanations and examples. I also want to go on a little tangent to explain **why** inner products are the similarity measure of choice in QML specifically.

Inner products measure both alignment and magnitude between vectors, specifically through relative phase and the norm (always positive). They also measure measurement similarity, so a perfect overlap $\braket{x_i|x_i}=1$ means that two quantum states will yield identical measurement outcomes across all bases. Both geometric and measurement similarities are satisfied, along with perservation of norm and alignment.

I know this is a lot, and you haven't seen a single line of code yet. But hang in there! 

# Primer on Tensors

Now we'e talking. Higher dimensions require higher order mathematical objects to represent them. Enter *Tensors.* Tensors are, simply speaking, generalizations of scalars, vectors, and matrices to higher dimensions. They can be thought of as multi-dimensional arrays that can represent data with multiple indices. Both `NumPy` and `PyTorch` have built-in support for tensors, it's just that in `NumPy` they're (somewhat misleadingly) called `ndarrays`, while in `PyTorch` they're actually called `Tensors`.

First of all, let's get PyTorch installed. There's a regular CPU version, and a GPU-accelerated version. If you have a compatible NVIDIA GPU, I recommend installing the GPU version for better performance. You can find the installation instructions on the [official PyTorch website](https://pytorch.org/get-started/locally/). Restarting the Jupyter kernel after installation is recommended to ensure that the newly installed packages are properly loaded.

## GPU-accelerated version (if you have an NVIDIA GPU compatible with CUDA 11.8+):

In [1]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

^C
Note: you may need to restart the kernel to use updated packages.


## Just CPU version:

In [ ]:
%pip install torch torchvision

Now, import torch and other required libraries, as well as the usual Qiskit components:

In [2]:
import torch as torch, torch.nn as nn, time

ModuleNotFoundError: No module named 'torch'

In [ ]:
import numpy as np, matplotlib.pyplot as plt
plt.style.use(['science', 'notebook', 'grid', 'dark_background'])

# OTHER QISKIT IMPORTS

from qiskit import *
from qiskit.visualization import *
from qiskit_ibm_runtime import *
from qiskit_aer import *

# DRAW STYLE (default mpl dark), check https://quantum.cloud.ibm.com/docs/en/guides/visualize-circuits and 
# https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.visualization.circuit_drawer for more styles

def draw_circuit(qc):
    return qc.draw(output = 'mpl', style = 'iqp-dark')